<a href="https://colab.research.google.com/github/YasiruMM/Medicine-Prediction-Grp-22/blob/Out-stock-predict/Flask_OOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-cors pyngrok joblib pandas numpy


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import joblib
from flask_cors import CORS  # To allow frontend access
from pyngrok import ngrok
import threading



In [ ]:
!ngrok config add-authtoken 2tobmc7DDP8dlEKWTZyC0BOSf4m_6ha1VZVY5Xd7KJXWCp1QY


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
OOS_app = Flask(__name__)
CORS(OOS_app)  # Enable CORS for frontend access

# Load the trained Random Forest model
rf_model = joblib.load("/content/drive/My Drive/DSGP/RandomForest_OOS.pkl")

# File paths for datasets
features_file = "/content/drive/My Drive/DSGP/NoiseHandled1_MediTrack_Dataset.csv"
predictions_file = "/content/drive/My Drive/DSGP/XGBoost_Predictions.csv"


In [ ]:
def generate_merged_dataset():
    try:
        # Load datasets
        df_features = pd.read_csv(features_file)
        df_predictions = pd.read_csv(predictions_file)

        # Transform XGBoost Predictions to Log Scale
        prediction_cols = ["Prediction 1", "Prediction 2", "Prediction 3",
                           "Prediction 4", "Prediction 5", "Prediction 6"]
        df_predictions[prediction_cols] = np.log1p(df_predictions[prediction_cols])

        # Keep Only Forecasted Drugs (Drugs that exist in Predictions)
        df_filtered = df_features[df_features["Drug Name"].isin(df_predictions["Drug Name"])]


        # Aggregate numeric features before merging using a dictionary.
        agg_dict = {
            "Retail Price": "mean",         # Average retail price
            "Purchase Price": "mean",       # Average purchase price
            "Sales": "sum",                 # Total sales
            "Mean Sales": "mean",           # Mean sales
            "Buffer Stock": "mean",         # Average buffer stock
            "Sales_LOESS_Date": "first"  # Use first occurrence for Sales_LOESS_Date
        }
        df_features_agg = df_filtered.groupby("Drug Name", as_index=False).agg(agg_dict)

        # Merge aggregated features with predictions
        df_merged = df_features_agg.merge(df_predictions, on="Drug Name", how="left")

        # Drop duplicate Disease Category column and rename
        df_merged.drop(columns=["Disease Category_y"], inplace=True, errors="ignore")
        df_merged.rename(columns={"Disease Category_x": "Disease Category"}, inplace=True)

        # Remove duplicates based on predictions
        df_merged = df_merged.drop_duplicates(
            subset=["Drug Name", "Prediction 1", "Prediction 2",
                    "Prediction 3", "Prediction 4", "Prediction 5", "Prediction 6"],
            keep="first"
        )

        # Compute log-transformed features:
        df_merged["Log_Sales"] = np.log1p(df_merged["Sales"])
        df_merged["Log_Retail_Price"] = np.log1p(df_merged["Retail Price"])
        df_merged["Log_Purchase_Price"] = np.log1p(df_merged["Purchase Price"])
        df_merged["Log_Buffer_Stock"] = np.log1p(df_merged["Buffer Stock"])

        # Assign risk factors based on Disease Category
        loss_factors = {
            'Cardiovascular': 0.20,
            'Diabetes': 0.25,
            'Cholesterol': 0.15
        }
        df_merged['Loss Factor'] = df_merged['Disease Category'].map(loss_factors).fillna(0.15)
        df_merged['Loss Quantity'] = df_merged['Loss Factor'] * df_merged["Log_Sales"]
        df_merged['Loss Quantity'] = df_merged['Loss Quantity'].apply(lambda x: max(0, x))

        # Drop time-related features (if any)
        df_merged.drop(columns=['Date', 'Month', 'Year'], inplace=True, errors="ignore")

        # Preserve Original Drug Name for API filtering
        df_merged["Original Drug Name"] = df_merged["Drug Name"]

        # One-Hot Encode categorical features: Disease Category and Drug Name
        df_merged = pd.get_dummies(df_merged, columns=['Disease Category', 'Drug Name'], drop_first=True)

        return df_merged

    except Exception as e:
        return None, str(e)


In [ ]:
@OOS_app.route('/get-drug-names', methods=['GET'])
def get_drug_names():
    try:
        df_merged = generate_merged_dataset()
        if df_merged is None:
            return jsonify({"error": "Failed to generate dataset!"})
        if "Original Drug Name" not in df_merged.columns:
            return jsonify({"error": "'Original Drug Name' column not found!"})

        available_drugs = df_merged["Original Drug Name"].unique().tolist()
        return jsonify({"drug_names": available_drugs})

    except Exception as e:
        return jsonify({"error": str(e)})


In [ ]:
@OOS_app.route('/predict-oos', methods=['POST'])
def predict_oos():
    try:
        user_input = request.get_json()
        user_drug = user_input.get("drug_name")
        user_loss = user_input.get("loss_quantity")

        df_merged = generate_merged_dataset()
        if df_merged is None:
            return jsonify({"error": "Failed to generate dataset!"})
        if "Original Drug Name" not in df_merged.columns:
            return jsonify({"error":" 'Original Drug Name' column not found!"})

        if user_drug not in df_merged["Original Drug Name"].values:
            return jsonify({"error": f"Drug '{user_drug}' not found!"})
        # injecting User inputs
        X_user = df_merged[df_merged["Original Drug Name"] == user_drug].copy()
        X_user["Loss Quantity"] = np.log1p(user_loss)  # Convert to log scale
        features = ['Sales_LOESS_Date', 'Log_Sales', 'Log_Buffer_Stock', 'Log_Retail_Price',
                    'Log_Purchase_Price', 'Loss Quantity'] + list(df_merged.columns[df_merged.columns.str.startswith(('Disease Category_', 'Drug Name_'))])

        X_user = X_user[features]

        y_pred_log = rf_model.predict(X_user)
        y_pred_log=y_pred_log.reshape(-1)
        y_pred_exp = np.expm1(y_pred_log)
        # row wise adjusting the predictions
        xgb_original = df_merged[df_merged["Original Drug Name"] == user_drug][["Prediction 1", "Prediction 2", "Prediction 3", "Prediction 4", "Prediction 5", "Prediction 6"]].values.flatten()
        # Use the trained Random Forest model to predict the adjusted demand (risk quantity)
        adjusted_predictions = y_pred_exp.flatten()  # Use ML-based predictions instead of simple subtraction


       #Compare adjusted vs original to get risk
        shortage_risk =  np.array(xgb_original)-np.array(adjusted_predictions)
        shortage_risk_levels = ["HIGH" if val > 0 else "LOW" for val in shortage_risk]


        result = {
            "drug_name": user_drug,
            "loss_quantity": user_loss,
            "original_predictions": xgb_original.tolist(),
            "adjusted_predictions": y_pred_exp.flatten().tolist(),
            "shortage_risk_levels": shortage_risk_levels
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({"error": str(e)})


In [ ]:
# Start Flask in a background thread
def run_flask():
    OOS_app.run(port=5000)

import threading
thread = threading.Thread(target=run_flask)
thread.start()

# Get the new ngrok URL
public_url = ngrok.connect(5000).public_url
print(f"Your Flask API is available at: {public_url}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Your Flask API is available at: https://ca7d-35-227-120-109.ngrok-free.app
